In [1]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [2]:
%%time
#"SFWRedheads","sfwpetite","SFWNextDoorGirls",
# subs = ["CityPorn","EarthPorn","memes","spaceporn","trippinthroughtime","fatsquirrelhate","oldladiesbakingpies","itookapicture", "celebrities"
subs = ["wallstreetbets"]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

extant_data = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
display(extant_data)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1000cej,CityPorn,OtterlyFoxy,New York in the fog,cars are parked on the side of the road in the...,7a8d96e378c15c8ab8440ac311f12c11,/r/CityPorn/comments/1000cej/new_york_in_the_fog/,https://i.redd.it/4emw5uldib9a1.jpg,4emw5uldib9a1.jpg,D:\data\images\CityPorn\4emw5uldib9a1.jpg,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,False
1,1000glf,AmIhotAF,toolate_sharkbait,Just looking for entertainment,blonde woman with blonde hair and tattoos on h...,e554c1ed7ffa2740436ac082068b2824,/r/AmIhotAF/comments/1000glf/just_looking_for_...,https://i.redd.it/4xyb1vgbjb9a1.jpg,4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,False
2,1000j1n,greentext,trent8051,Anon wants Elon cut,a man with a beard and a beard sitting in fron...,1dec3dabb5e46cde01855d06089c287a,/r/greentext/comments/1000j1n/anon_wants_elon_...,https://i.redd.it/3mewbe0wjb9a1.jpg,3mewbe0wjb9a1.jpg,D:\data\images\greentext\3mewbe0wjb9a1.jpg,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,False
3,1000mjs,spaceporn,MorningStar_imangi,Northern Lights above Lofoten,a view of a view of a large green and purple a...,2c39ce1290fba541abd0b004b09da6b2,/r/spaceporn/comments/1000mjs/northern_lights_...,https://i.redd.it/7s5aafaqkb9a1.jpg,7s5aafaqkb9a1.jpg,/data/images/spaceporn/7s5aafaqkb9a1.jpg,/data/images/spaceporn/thumbnail/7s5aafaqkb9a1...,True,False
4,1000qpd,spaceporn,MorningStar_imangi,Viking Lights,a scene of a boat is sitting on the shore of a...,0f72de47c69ff50eca5fa3990215f4ac,/r/spaceporn/comments/1000qpd/viking_lights/,https://i.redd.it/abojw7lqlb9a1.jpg,abojw7lqlb9a1.jpg,/data/images/spaceporn/abojw7lqlb9a1.jpg,/data/images/spaceporn/thumbnail/abojw7lqlb9a1...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21741,11j03l0,celebrities,Jzk592,Gal Gadot,arafed image of a woman in a scarf around her ...,937ab8bcc41f39055217d64c38a447d9,/r/celebrities/comments/11j03l0/gal_gadot/,https://i.redd.it/s1ik1llgqve81.jpg,937ab8bcc41f39055217d64c38a447d9.jpg,D:\data\images\celebrities\937ab8bcc41f3905521...,D:\data\images\celebrities\thumbnail\937ab8bcc...,True,False
21742,11ns444,celebrities,ConnectionOk8413,Amy Adams,a woman in a black leather jacket and tie dyes...,641520ae16f4aabd494172f80267ebed,/r/celebrities/comments/11ns444/amy_adams/,https://i.redd.it/niul7blwjxma1.jpg,641520ae16f4aabd494172f80267ebed.jpg,D:\data\images\celebrities\641520ae16f4aabd494...,D:\data\images\celebrities\thumbnail\641520ae1...,True,False
21743,11nrqmo,celebrities,Ahenze85,Kate Beckinsale,arafed image of a woman sitting in a chair wit...,85ea0e7a20268adaaefd8cbc31868647,/r/celebrities/comments/11nrqmo/kate_beckinsale/,https://i.redd.it/qihkgqruyyma1.jpg,85ea0e7a20268adaaefd8cbc31868647.jpg,D:\data\images\celebrities\85ea0e7a20268adaaef...,D:\data\images\celebrities\thumbnail\85ea0e7a2...,True,False
21744,11nmtns,celebrities,mikere24,Madelaine Petsch,a woman in a sheer - lined dress with a sheer ...,296b6a7b159560e2811f41a4d659050e,/r/celebrities/comments/11nmtns/madelaine_petsch/,https://i.redd.it/kifsl6yywxma1.jpg,296b6a7b159560e2811f41a4d659050e.jpg,D:\data\images\celebrities\296b6a7b159560e2811...,D:\data\images\celebrities\thumbnail\296b6a7b1...,True,False


CPU times: total: 922 ms
Wall time: 2.4 s


In [4]:
reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA',
								  user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

api: PushshiftAPI = PushshiftAPI(praw=reddit, num_workers=12)
start_date: int = int(datetime.strptime('2022-01-01', '%Y-%m-%d').timestamp())
end_date: int = int(datetime.today().strftime('%Y-%m-%d').timestamp())

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [5]:
def filter_by_image_post(item) -> bool:
	return not (item['selftext'].__contains__('[removed]') or item['selftext'].__contains__('[deleted]') and not item[
		'url'].endswith('.jpg'))

In [6]:
def fetch_image(x: object) -> object:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		# print(e)
		return ""

In [7]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		# print(e)
		return ""
	finally:
		img.close()

In [8]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		# print(e)
		return False

In [9]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		# print(e)
		return ""

In [10]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		return ""
	finally:
		pass

In [11]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		# print(e)
		return ""

In [12]:
%%time

all_posts = []

for item in subs:
	posts = api.search_submissions(search_window=365, subreddit=item, limit=1000, filter_fn=filter_by_image_post, mem_safe=True, cache_dir='D:\\cache\\')
	[ all_posts.append(post) for post in posts ]

CPU times: total: 2.72 s
Wall time: 30.1 s


In [13]:
%%time

df = pd.DataFrame([item for item in all_posts])
display(df)

,comment_limit,comment_sort,_reddit,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,...,is_video,_fetched,_comments_by_id,is_gallery,media_metadata,gallery_data,post_hint,preview,live_audio,poll_data
0,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_czgquvwj,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_7swvtua2,False,None,0,...,False,False,{},True,None,None,NaN,NaN,NaN,NaN
2,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_h0pgvyzr,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_5i7a4jyal,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_2f59r55a,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_a2xnhr02,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_135x51,False,None,0,...,False,False,{},NaN,NaN,NaN,image,{'images': [{'source': {'url': 'https://previe...,NaN,NaN
517,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_1n9p9vz7,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,2048,confidence,<praw.reddit.Reddit object at 0x000001E80E0F30A0>,None,wallstreetbets,,t2_7d7h6j85,False,None,0,...,False,False,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN


CPU times: total: 125 ms
Wall time: 133 ms


In [14]:
%%time

initial = pd.DataFrame({}, columns=schema.names)
initial['id'] = df['id']
initial['subreddit'] = df['subreddit']
initial['author'] = df['author']
initial['title'] = df['title']
initial['caption'] = ""
initial['hash'] = ""
initial['permalink'] = df['permalink']
initial['original_url'] = df['url']
initial['image_name'] = ""
initial['path'] = ""
initial['thumbnail_path'] = ""
initial['exists'] = False
initial['curated'] = False

filtered_initial = initial.where(initial['original_url'].str.endswith('.jpg')).dropna(how='all').reset_index().drop('index', axis=1)

display(filtered_initial)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1165ays,wallstreetbets,n618ft,What banks and other assets will be picked off...,,,/r/wallstreetbets/comments/1165ays/what_banks_...,https://i.redd.it/y5owo797c5ja1.jpg,,,,False,False
1,1162tt3,wallstreetbets,backtosky,The struggle is real,,,/r/wallstreetbets/comments/1162tt3/the_struggl...,https://i.redd.it/v4u2azo8k4ja1.jpg,,,,False,False
2,1161lt4,wallstreetbets,Sirachacopter,"Apes everywhere - Carrizo gorge, CA",,,/r/wallstreetbets/comments/1161lt4/apes_everyw...,https://i.imgur.com/5MGcjws.jpg,,,,False,False
3,1161k6p,wallstreetbets,Sirachacopter,"Carrizo Gorge, CA - Ok, which one of you apes ...",,,/r/wallstreetbets/comments/1161k6p/carrizo_gor...,https://i.imgur.com/dZ03inb.jpg,,,,False,False
4,1160kjj,wallstreetbets,SharkSapphire,$TSLA Cybertruck 2.0,,,/r/wallstreetbets/comments/1160kjj/tsla_cybert...,https://i.redd.it/6mf7m5qmx3ja1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,10365m3,wallstreetbets,ToiletHandl,"I sold at 32% gain, but it’s still going up.. ...",,,/r/wallstreetbets/comments/10365m3/i_sold_at_3...,https://i.redd.it/bv84lwzv43aa1.jpg,,,,False,False
201,1036071,wallstreetbets,Bethanysteele72,Bite me,,,/r/wallstreetbets/comments/1036071/bite_me/,https://i.redd.it/vybemlat33aa1.jpg,,,,False,False
202,1035vqc,wallstreetbets,kevinyyh2020,SPY PUTS GAIN. 100k achieved finally.,,,/r/wallstreetbets/comments/1035vqc/spy_puts_ga...,https://i.redd.it/9ctzlozv23aa1.jpg,,,,False,False
203,1035nhr,wallstreetbets,ElNinoDelGato,Going short on the SPY again.,,,/r/wallstreetbets/comments/1035nhr/going_short...,https://i.redd.it/rojt96f713aa1.jpg,,,,False,False


CPU times: total: 78.1 ms
Wall time: 86.7 ms


In [15]:
%%time

new_entries = filtered_initial.where(~filtered_initial['id'].isin(extant_data['id'])).dropna(how='all').reset_index().drop('index', axis=1)

display(new_entries)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1165ays,wallstreetbets,n618ft,What banks and other assets will be picked off...,,,/r/wallstreetbets/comments/1165ays/what_banks_...,https://i.redd.it/y5owo797c5ja1.jpg,,,,False,False
1,1162tt3,wallstreetbets,backtosky,The struggle is real,,,/r/wallstreetbets/comments/1162tt3/the_struggl...,https://i.redd.it/v4u2azo8k4ja1.jpg,,,,False,False
2,1161lt4,wallstreetbets,Sirachacopter,"Apes everywhere - Carrizo gorge, CA",,,/r/wallstreetbets/comments/1161lt4/apes_everyw...,https://i.imgur.com/5MGcjws.jpg,,,,False,False
3,1161k6p,wallstreetbets,Sirachacopter,"Carrizo Gorge, CA - Ok, which one of you apes ...",,,/r/wallstreetbets/comments/1161k6p/carrizo_gor...,https://i.imgur.com/dZ03inb.jpg,,,,False,False
4,1160kjj,wallstreetbets,SharkSapphire,$TSLA Cybertruck 2.0,,,/r/wallstreetbets/comments/1160kjj/tsla_cybert...,https://i.redd.it/6mf7m5qmx3ja1.jpg,,,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,10365m3,wallstreetbets,ToiletHandl,"I sold at 32% gain, but it’s still going up.. ...",,,/r/wallstreetbets/comments/10365m3/i_sold_at_3...,https://i.redd.it/bv84lwzv43aa1.jpg,,,,False,False
201,1036071,wallstreetbets,Bethanysteele72,Bite me,,,/r/wallstreetbets/comments/1036071/bite_me/,https://i.redd.it/vybemlat33aa1.jpg,,,,False,False
202,1035vqc,wallstreetbets,kevinyyh2020,SPY PUTS GAIN. 100k achieved finally.,,,/r/wallstreetbets/comments/1035vqc/spy_puts_ga...,https://i.redd.it/9ctzlozv23aa1.jpg,,,,False,False
203,1035nhr,wallstreetbets,ElNinoDelGato,Going short on the SPY again.,,,/r/wallstreetbets/comments/1035nhr/going_short...,https://i.redd.it/rojt96f713aa1.jpg,,,,False,False


CPU times: total: 46.9 ms
Wall time: 46.9 ms


In [16]:
%%time

new_entries_1 = new_entries.copy()

tqdm.pandas(desc="Fetch-Image")
with ProgressBar():
	new_entries_1['path'] = new_entries_1.progress_apply(lambda x: fetch_image(x), axis=1)

tqdm.pandas(desc="Get-Image-Name")
with ProgressBar():
	new_entries_1['image_name'] = new_entries_1.progress_apply(lambda x: get_image_name(x), axis=1)

tqdm.pandas(desc="Split-Hash")
with ProgressBar():
	new_entries_1['hash'] = new_entries_1.progress_apply(lambda x: split_hash(x), axis=1)

display(new_entries_1)

Split-Hash: 100%|██████████| 205/205 [00:00<00:00, 6560.15it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1165ays,wallstreetbets,n618ft,What banks and other assets will be picked off...,,772ccdf2d1730340471ff107b0f14c40,/r/wallstreetbets/comments/1165ays/what_banks_...,https://i.redd.it/y5owo797c5ja1.jpg,772ccdf2d1730340471ff107b0f14c40.jpg,D:\data\images\wallstreetbets\772ccdf2d1730340...,,False,False
1,1162tt3,wallstreetbets,backtosky,The struggle is real,,f7ca83705918c420f9d544af776d6c1e,/r/wallstreetbets/comments/1162tt3/the_struggl...,https://i.redd.it/v4u2azo8k4ja1.jpg,f7ca83705918c420f9d544af776d6c1e.jpg,D:\data\images\wallstreetbets\f7ca83705918c420...,,False,False
2,1161lt4,wallstreetbets,Sirachacopter,"Apes everywhere - Carrizo gorge, CA",,6889e2316257e41c60186c4f5edfa3e1,/r/wallstreetbets/comments/1161lt4/apes_everyw...,https://i.imgur.com/5MGcjws.jpg,6889e2316257e41c60186c4f5edfa3e1.jpg,D:\data\images\wallstreetbets\6889e2316257e41c...,,False,False
3,1161k6p,wallstreetbets,Sirachacopter,"Carrizo Gorge, CA - Ok, which one of you apes ...",,6889e2316257e41c60186c4f5edfa3e1,/r/wallstreetbets/comments/1161k6p/carrizo_gor...,https://i.imgur.com/dZ03inb.jpg,6889e2316257e41c60186c4f5edfa3e1.jpg,D:\data\images\wallstreetbets\6889e2316257e41c...,,False,False
4,1160kjj,wallstreetbets,SharkSapphire,$TSLA Cybertruck 2.0,,fd2fdbf24ccb6f22068e139a7082b87a,/r/wallstreetbets/comments/1160kjj/tsla_cybert...,https://i.redd.it/6mf7m5qmx3ja1.jpg,fd2fdbf24ccb6f22068e139a7082b87a.jpg,D:\data\images\wallstreetbets\fd2fdbf24ccb6f22...,,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,10365m3,wallstreetbets,ToiletHandl,"I sold at 32% gain, but it’s still going up.. ...",,4cd75ac12563baded80ac7a79152ae1e,/r/wallstreetbets/comments/10365m3/i_sold_at_3...,https://i.redd.it/bv84lwzv43aa1.jpg,4cd75ac12563baded80ac7a79152ae1e.jpg,D:\data\images\wallstreetbets\4cd75ac12563bade...,,False,False
201,1036071,wallstreetbets,Bethanysteele72,Bite me,,39a441be43bebb6c64301069c71c2e84,/r/wallstreetbets/comments/1036071/bite_me/,https://i.redd.it/vybemlat33aa1.jpg,39a441be43bebb6c64301069c71c2e84.jpg,D:\data\images\wallstreetbets\39a441be43bebb6c...,,False,False
202,1035vqc,wallstreetbets,kevinyyh2020,SPY PUTS GAIN. 100k achieved finally.,,a50173fd92b92f08d1f96b6cf921788f,/r/wallstreetbets/comments/1035vqc/spy_puts_ga...,https://i.redd.it/9ctzlozv23aa1.jpg,a50173fd92b92f08d1f96b6cf921788f.jpg,D:\data\images\wallstreetbets\a50173fd92b92f08...,,False,False
203,1035nhr,wallstreetbets,ElNinoDelGato,Going short on the SPY again.,,9edbc56e99449ad01b07e632396d18c3,/r/wallstreetbets/comments/1035nhr/going_short...,https://i.redd.it/rojt96f713aa1.jpg,9edbc56e99449ad01b07e632396d18c3.jpg,D:\data\images\wallstreetbets\9edbc56e99449ad0...,,False,False


CPU times: total: 25.2 s
Wall time: 1min 8s


In [17]:
%%time

new_entries_2 = new_entries_1.copy()

tqdm.pandas(desc="Create-Thumbnail")
with ProgressBar():
	new_entries_2['thumbnail_path'] = new_entries_2.progress_apply(lambda x: make_thumbnail(x), axis=1)

tqdm.pandas(desc="Checking-Exists")
with ProgressBar():
	new_entries_2['exists'] = new_entries_2.progress_apply(lambda x: exists(x), axis=1)

display(new_entries_2)

Checking-Exists: 100%|██████████| 205/205 [00:00<00:00, 3279.99it/s]


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,thumbnail_path,exists,curated
0,1165ays,wallstreetbets,n618ft,What banks and other assets will be picked off...,,772ccdf2d1730340471ff107b0f14c40,/r/wallstreetbets/comments/1165ays/what_banks_...,https://i.redd.it/y5owo797c5ja1.jpg,772ccdf2d1730340471ff107b0f14c40.jpg,D:\data\images\wallstreetbets\772ccdf2d1730340...,D:\data\images\wallstreetbets\thumbnail\772ccd...,True,False
1,1162tt3,wallstreetbets,backtosky,The struggle is real,,f7ca83705918c420f9d544af776d6c1e,/r/wallstreetbets/comments/1162tt3/the_struggl...,https://i.redd.it/v4u2azo8k4ja1.jpg,f7ca83705918c420f9d544af776d6c1e.jpg,D:\data\images\wallstreetbets\f7ca83705918c420...,D:\data\images\wallstreetbets\thumbnail\f7ca83...,True,False
2,1161lt4,wallstreetbets,Sirachacopter,"Apes everywhere - Carrizo gorge, CA",,6889e2316257e41c60186c4f5edfa3e1,/r/wallstreetbets/comments/1161lt4/apes_everyw...,https://i.imgur.com/5MGcjws.jpg,6889e2316257e41c60186c4f5edfa3e1.jpg,D:\data\images\wallstreetbets\6889e2316257e41c...,D:\data\images\wallstreetbets\thumbnail\6889e2...,True,False
3,1161k6p,wallstreetbets,Sirachacopter,"Carrizo Gorge, CA - Ok, which one of you apes ...",,6889e2316257e41c60186c4f5edfa3e1,/r/wallstreetbets/comments/1161k6p/carrizo_gor...,https://i.imgur.com/dZ03inb.jpg,6889e2316257e41c60186c4f5edfa3e1.jpg,D:\data\images\wallstreetbets\6889e2316257e41c...,D:\data\images\wallstreetbets\thumbnail\6889e2...,True,False
4,1160kjj,wallstreetbets,SharkSapphire,$TSLA Cybertruck 2.0,,fd2fdbf24ccb6f22068e139a7082b87a,/r/wallstreetbets/comments/1160kjj/tsla_cybert...,https://i.redd.it/6mf7m5qmx3ja1.jpg,fd2fdbf24ccb6f22068e139a7082b87a.jpg,D:\data\images\wallstreetbets\fd2fdbf24ccb6f22...,D:\data\images\wallstreetbets\thumbnail\fd2fdb...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,10365m3,wallstreetbets,ToiletHandl,"I sold at 32% gain, but it’s still going up.. ...",,4cd75ac12563baded80ac7a79152ae1e,/r/wallstreetbets/comments/10365m3/i_sold_at_3...,https://i.redd.it/bv84lwzv43aa1.jpg,4cd75ac12563baded80ac7a79152ae1e.jpg,D:\data\images\wallstreetbets\4cd75ac12563bade...,D:\data\images\wallstreetbets\thumbnail\4cd75a...,True,False
201,1036071,wallstreetbets,Bethanysteele72,Bite me,,39a441be43bebb6c64301069c71c2e84,/r/wallstreetbets/comments/1036071/bite_me/,https://i.redd.it/vybemlat33aa1.jpg,39a441be43bebb6c64301069c71c2e84.jpg,D:\data\images\wallstreetbets\39a441be43bebb6c...,D:\data\images\wallstreetbets\thumbnail\39a441...,True,False
202,1035vqc,wallstreetbets,kevinyyh2020,SPY PUTS GAIN. 100k achieved finally.,,a50173fd92b92f08d1f96b6cf921788f,/r/wallstreetbets/comments/1035vqc/spy_puts_ga...,https://i.redd.it/9ctzlozv23aa1.jpg,a50173fd92b92f08d1f96b6cf921788f.jpg,D:\data\images\wallstreetbets\a50173fd92b92f08...,D:\data\images\wallstreetbets\thumbnail\a50173...,True,False
203,1035nhr,wallstreetbets,ElNinoDelGato,Going short on the SPY again.,,9edbc56e99449ad01b07e632396d18c3,/r/wallstreetbets/comments/1035nhr/going_short...,https://i.redd.it/rojt96f713aa1.jpg,9edbc56e99449ad01b07e632396d18c3.jpg,D:\data\images\wallstreetbets\9edbc56e99449ad0...,D:\data\images\wallstreetbets\thumbnail\9edbc5...,True,False


CPU times: total: 20.8 s
Wall time: 31.1 s


In [18]:
%%time

blip = BlipCaption(1)

CPU times: total: 1min 2s
Wall time: 1min 21s


In [ ]:
%%time

new_entries_final = new_entries_2.copy()
torch.cuda.empty_cache()

with pbar:
	ddf = dd.from_pandas(new_entries_final, npartitions=6)
	new_entries_final['caption'] = ddf.apply(lambda x: caption_image(x, blip), meta=('str', object), axis=1).compute()

torch.cuda.empty_cache()

display(new_entries_final)

[                                        ] | 0% Completed | 4.93 ms

global:   0%|          | 0/6 [00:00<?, ?it/s]

[                                        ] | 0% Completed | 28.28 sms

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[######                                  ] | 16% Completed | 198.08 s

D:\workspaces\General\venv\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[######                                  ] | 16% Completed | 230.87 s

In [ ]:
%%time

tqdm.pandas(desc="Filtering")
writeable_entries = new_entries_final\
	.where(~new_entries_final['id'].isin(extant_data['id']))\
	.dropna(how='all')\
	.reset_index()\
	.drop('index', axis=1)

display(writeable_entries)

In [ ]:
%%time

combined_result = pd.concat([extant_data, writeable_entries])

display(combined_result)

In [ ]:
combined_result.reindex()
combined_result.to_csv('processed_raw_data.csv', index=False)
f = AzureFileStorageAdapter('data').get_file_storage()
f.put('processed_raw_data.csv','data/processed_raw_data.csv', overwrite=True)

In [ ]:
final = pd.read_csv(f.open('data/processed_raw_data.csv'))
final.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [ ]:
final = pd.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system)
display(final)